# Low-Level Design (LLD) of a ATM System

### 1. Requirements

1. **Authenticate Users**:
    - The system should authenticate users based on their card number and PIN before performing any operations.

2. **Deposit Money**:
    - The system should allow users to deposit money into their accounts.

3. **Withdraw Money:**:
    - The system should allow users to withdraw money, ensuring sufficient funds are available both in the account and in the ATM

4. **Account Balance**:
    - The system should allow users to check their account balance.

5. **Transaction History:**:
    - The system should allow users to view their past transactions.

6. **ATM Cash Management**:
    -  The ATM should handle its cash availability and notify when the cash is low.

7. **Security**:
    - Secure transactions should be ensured through proper authentication, transaction validation, and encryption

---

### 2. Constraints

1. The system should check if the ATM has sufficient cash before allowing withdrawals.
2. The system should validate that the account has sufficient balance for withdrawal.
3. The system must handle concurrent transactions (multiple users) while ensuring that one user’s operation doesn't interfere with another
4. Users must authenticate using their card number and PIN.

---

### 3. Identify Entities

1. **ATM**:
    - Manages ATM operations, including cash availability, handling deposits/withdrawals, and user authentication.

2. **Account**:
    - Represents the user's account, containing account details like balance, and provides methods to deposit and withdraw funds.

3. **Transaction**:
    - Represents a transaction such as deposit, withdrawal, or balance inquiry. It contains information like the transaction type, amount, status, and timestamp.

4. **User**:
    - Represents a user of the ATM with personal details and a link to their account.

5. **Bank**:
    - Manages users and accounts, ensures users' funds are available, and supports authentication.

6. **TransactionHistory**:
    - Stores all the transactions of a user for tracking and auditing purposes.

7. **Security**:
    - Handles encryption and PIN verification for secure access.

### 4. Class Design

#### 4.1. Security Class

In [1]:
import hashlib


class Security:
    @staticmethod
    def hash_pin(pin):
        """Hash PIN using SHA-256 for security"""
        return hashlib.sha256(pin.encode()).hexdigest()

    @staticmethod
    def verify_pin(input_pin, stored_hashed_pin):
        """Verify PIN against stored hash"""
        return Security.hash_pin(input_pin) == stored_hashed_pin


#### 4.2. User Class

In [4]:
class User:
    def __init__(self, user_id, name, card_number, pin, account):
        self.user_id = user_id
        self.name = name
        self.card_number = card_number
        self.hashed_pin = Security.hash_pin(pin)  # Secure PIN storage
        self.account = account

    def authenticate(self, pin):
        """Authenticate user by verifying PIN"""
        return Security.verify_pin(pin, self.hashed_pin)

    def check_balance(self):
        """Check account balance"""
        return self.account.balance

    def get_transaction_history(self):
        """Retrieve last 5 transactions"""
        return self.account.transaction_history.get_recent_transactions()


#### 4.3. Account Class

In [6]:
import threading
from datetime import datetime


class Account:
    def __init__(self, account_number, balance=0):
        self.account_number = account_number
        self.balance = balance
        self.transaction_history = TransactionHistory()
        self.lock = threading.Lock()  # Lock for thread safety

    def deposit(self, amount):
        """Deposit money into account (Thread-safe)"""
        with self.lock:
            if amount <= 0:
                raise InvalidTransactionException("Deposit amount must be positive.")

            self.balance += amount
            transaction = Transaction("Deposit", amount, "Success", datetime.now())
            self.transaction_history.add_transaction(transaction)
            return transaction

    def withdraw(self, amount):
        """Withdraw money from account (Thread-safe)"""
        with self.lock:
            if amount > self.balance:
                raise InsufficientFundsException("Insufficient funds in the account.")

            self.balance -= amount
            transaction = Transaction("Withdrawal", amount, "Success", datetime.now())
            self.transaction_history.add_transaction(transaction)
            return transaction


#### 4.4. Bank Class

In [8]:
class Bank:
    users = {}

    @classmethod
    def add_user(cls, user):
        cls.users[user.card_number] = user

    @classmethod
    def get_user_by_card(cls, card_number):
        return cls.users.get(card_number, None)

#### 4.5. Transaction Class

In [10]:
class Transaction:
    def __init__(self, transaction_type, amount, status, timestamp):
        self.transaction_type = transaction_type
        self.amount = amount
        self.status = status
        self.timestamp = timestamp

    def __str__(self):
        return f"{self.transaction_type}: - {self.amount} at {self.timestamp}"

#### 4.6. TransactionHistory Class

In [13]:
class TransactionHistory:
    def __init__(self):
        self.transactions = []

    def add_transaction(self, transaction):
        """Add a transaction to history"""
        self.transactions.append(transaction)

    def get_recent_transactions(self, count=5):
        """Get last 'count' transactions"""
        return self.transactions[-count:]

#### 4.7. ATM Class

In [15]:
class ATM:
    def __init__(self, atm_id, location, cash_limit=10000):
        self.atm_id = atm_id
        self.location = location
        self.cash_limit = cash_limit
        self.current_cash = cash_limit
        self.lock = threading.Lock()  # Lock for ATM-wide transactions
        self.authenticated_users = {}  # Store authenticated users

    def authenticate_user(self, card_number, pin):
        """Authenticate user and store session"""
        user = Bank.get_user_by_card(card_number)
        if user and user.authenticate(pin):
            self.authenticated_users[user.user_id] = True
            return user
        else:
            raise AuthenticationException("Invalid card number or PIN")

    def is_authenticated(self, user):
        """Check if user is authenticated"""
        return self.authenticated_users.get(user.user_id, False)

    def deposit_money(self, user, amount):
        """Deposit money into the user's account (Thread-safe)"""
        if not self.is_authenticated(user):
            raise AuthenticationException("User not authenticated.")

        with self.lock:
            user.account.deposit(amount)
            self.current_cash += amount
            return f"Deposited {amount} successfully."

    def withdraw_money(self, user, amount):
        """Withdraw money securely (Thread-safe)"""
        if not self.is_authenticated(user):
            raise AuthenticationException("User not authenticated.")

        with self.lock:
            if self.current_cash < amount:
                raise ATMOutOfCashException("ATM is out of cash.")

            withdrawn_amount = user.account.withdraw(amount)
            self.current_cash -= amount
            return f"Withdrawn {withdrawn_amount} successfully."

    def check_balance(self, user):
        """Check account balance"""
        if not self.is_authenticated(user):
            raise AuthenticationException("User not authenticated.")
        return f"Your balance is: {user.check_balance()}"

    def get_transaction_history(self, user):
        """Get last 5 transactions"""
        if not self.is_authenticated(user):
            raise AuthenticationException("User not authenticated.")
        return [str(tx) for tx in user.get_transaction_history()]

    def logout_user(self, user):
        """Logout user after transaction"""
        if user.user_id in self.authenticated_users:
            del self.authenticated_users[user.user_id]


### 5. Exception Handling

In [17]:
class ATMException(Exception):
    """Base class for all custom exceptions."""
    pass


class AuthenticationException(ATMException):
    """Raised when user authentication fails"""
    def __init__(self, message="Authentication Failed."):
        self.message = message
        super().__init__(self.message)


class InsufficientFundsException(ATMException):
    """Raised when account has insufficient balance"""
    def __init__(self, message="Account has insufficient balance"):
        self.message = message
        super().__init__(self.message)


class ATMOutOfCashException(ATMException):
    """Raised when the ATM has insufficient cash for withdrawal"""
    def __init__(self, message="ATM has insufficient cash for withdrawal."):
        self.message = message
        super().__init__(self.message)


class InvalidTransactionException(ATMException):
    """Raised when an invalid transaction is attempted"""
    def __init__(self, message="Invalid transaction."):
        self.message = message
        super().__init__(self.message)


### 6. Implementation

In [21]:
import threading

# Initialize ATM and Bank
atm = ATM(atm_id=1, location="Downtown", cash_limit=5000)
user_account = Account(account_number="12345", balance=1000)
user = User(user_id=1, name="John Doe", card_number="1234567890", pin="1234", account=user_account)
Bank.add_user(user)

# Authenticate User
user = atm.authenticate_user(card_number="1234567890", pin="1234")

# atm.withdraw_money(user, 200)
# atm.withdraw_money(user, 300)
# atm.deposit_money(user, 100)

# Define concurrent transactions
def concurrent_transaction(user, amount, transaction_type):
    try:
        if transaction_type == "withdraw":
            print(atm.withdraw_money(user, amount))
        elif transaction_type == "deposit":
            print(atm.deposit_money(user, amount))
    except Exception as e:
        print(f"Transaction Failed: {e}")

# Start threads
threads = []
for _ in range(3):
    t = threading.Thread(target=concurrent_transaction, args=(user, 200, "withdraw"))
    threads.append(t)

for _ in range(2):
    t = threading.Thread(target=concurrent_transaction, args=(user, 500, "deposit"))
    threads.append(t)

for t in threads:
    t.start()

for t in threads:
    t.join()

# Final Balance
print(f"Final Balance: {user.check_balance()}")
print(f"Final ATM Cash: {atm.current_cash}")


Withdrawn Withdrawal: - 200 at 2025-02-04 17:26:56.056408 successfully.
Withdrawn Withdrawal: - 200 at 2025-02-04 17:26:56.057247 successfully.
Withdrawn Withdrawal: - 200 at 2025-02-04 17:26:56.057923 successfully.
Deposited 500 successfully.
Deposited 500 successfully.
Final Balance: 1400
Final ATM Cash: 5400
